# WeatherPy - Hotel Search

### Dependencies and data

In [1]:
# Dependencies
import os
import requests
import gmaps
import numpy as np
import pandas as pd
from config import GMAPS_KEY

In [2]:
# Configure API key
gmaps.configure(api_key=GMAPS_KEY)

In [3]:
# Weather data
data_path = os.path.join('data', 'weather.csv')
weather_df = pd.read_csv(data_path)
weather_df.head(2)

,Id,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow
0,0,Garoowe,SO,8.4054,48.4845,2021-04-18 15:28:30,clear sky,89.92,30,16.42,6,0,0
1,1,Vendas Novas,PT,38.6771,-8.4579,2021-04-18 15:28:30,few clouds,73.40,46,12.66,20,0,0


### Find hotels near destinations within the user's preferred weather

In [4]:
# Lowest and highest `Max Temp` in the data
min_mt, max_mt = weather_df['Max Temp'].min(), weather_df['Max Temp'].max()
min_mt, max_mt

(-17.61, 107.6)

In [10]:
def user_input(prompt, min_temp=min_mt, max_temp=max_mt, binary=False):
    
    """
    Take user input for weather preferences.
    
    Parameters
    ----------
    prompt : str
        Input prompt
    min_temp : float, optional
        Minimum temperature in data, by default {min_mt}
    max_temp : float, optional
        Maximum temperature in data, by default {max_mt}
    binary : bool, optional
        Whether the input should be "yes" or "no" only, by default False
        
    Returns
    -------
    float
        User input
    """
    
    # Init user input and loop condition to False
    i = b = False
    
    # Get user input
    if binary: # yes/no input
        while i not in ['yes', 'no']: # continue until input is either "yes" or "no"
            i = input(prompt) # user input
        i = 1 if i == 'yes' else 0 # convert input to num
    else: 
        prompt += f'\tEnter a number between {min_temp} and {max_temp}: ' 
        while not b: # loop condition
            try:
                i = float(input(prompt)) # get user input as num
            except:
                continue
            if min_temp <= i <= max_temp: # check if user input is within proper temperature range
                b = True # end loop
            
    # Numeric input
    return i
        
    
# Get preferred weather
min_temp = user_input('1. What is the lowest temperature you would like at your destination?\n')
max_temp = user_input('2. What is the highest temperature you would like at your destination?\n', min_temp=min_temp)
rain = user_input('3. Would you like it to rain at your destination? (yes/no) ', binary=True)
snow = user_input('4. Would you like it to snow at your destination? (yes/no) ', binary=True)
min_temp, max_temp, rain, snow # enter 70, 90, yes, no

1. What is the lowest temperature you would like at your destination?
	Enter a number between -17.61 and 107.6: 70
2. What is the highest temperature you would like at your destination?
	Enter a number between 70.0 and 107.6: 90
3. Would you like it to rain at your destination? (yes/no) yes
4. Would you like it to snow at your destination? (yes/no) no


(70.0, 90.0, 1, 0)

In [ ]:
# Filter for max temp preference
pref_cities = weather.loc[(weather['Max Temp'] >= min_temp) & (weather['Max Temp'] <= max_temp)]

# Filter for rain preference
if rain:
    pref_cities = pref_cities[pref_cities['Rain'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Rain'] == 0]

# Filter for snow preference
if snow:
    pref_cities = pref_cities[pref_cities['Snow'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Snow'] == 0]
    
pref_cities.shape

In [ ]:
# Create new dataframe
hotels = pref_cities.copy()
hotels['Hotel'] = np.NaN # create new col for hotels

# Request url and params
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': config.GMAPS_API_KEY,
    'type': 'lodging', # hotels
    'radius': 5000 # meters
}

hotels.head(2)

In [ ]:
# Sample response
payload['location'] = f'{hotels.iloc[0, 3]},{hotels.iloc[0, 4]}' # first city in data
response = requests.get(base_url, params=payload)
response.json()

In [ ]:
# Indices of cities without results
i404 = []

# Find nearest hotel for each city
for idx, row in hotels.iterrows():
    payload['location'] = f'{row["Latitude"]},{row["Longitude"]}' # city coordinates
    try:
        response = requests.get(base_url, params=payload)
        response_json = response.json()
        hotels.loc[idx, 'Hotel'] = response_json['results'][0]['name']
    except:
        i404.append(idx)

print(len(i404))
hotels.loc[i404, 'City'].tolist()

In [ ]:
# Drop cities without results
hotels.dropna(inplace=True)
print(hotels.shape)
hotels.head(2)

### Temperature heatmap

In [ ]:
# For Jupyterlab
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

In [ ]:
# Locations and measurements
locations = hotels[['Latitude', 'Longitude']]
weights = hotels['Max Temp'].apply(lambda t: max(t, 0))

# Info
hotel_info = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
"""

# Layers
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=7, max_intensity=140)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[hotel_info.format(**row) for idx, row in hotels.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(mark_layer)
fig

In [ ]:
# Save hotel data
hotels.to_csv('data/hotels.csv', index=False)
pd.read_csv('data/hotels.csv').head(2)